In [28]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 18: Boiling Boulders ---</h2><p>You and the elephants finally reach fresh air. You've emerged near the base of a large volcano that seems to be actively erupting! Fortunately, the lava seems to be flowing away from you and toward the ocean.</p>
<p>Bits of lava are still being ejected toward you, so you're sheltering in the cavern exit a little longer. Outside the cave, you can see the lava landing in a pond and hear it loudly hissing as it solidifies.</p>
<p>Depending on the specific compounds in the lava and speed at which it cools, it might be forming <a href="https://en.wikipedia.org/wiki/Obsidian" target="_blank">obsidian</a>! The cooling rate should be based on the surface area of the lava droplets, so you take a quick scan of a droplet as it flies past you (your puzzle input).</p>
<p>Because of how quickly the lava is moving, the scan isn't very good; its resolution is quite low and, as a result, it approximates the shape of the lava droplet with <em>1x1x1 <span title="Unfortunately, you forgot your flint and steel in another dimension.">cubes</span> on a 3D grid</em>, each given as its <code>x,y,z</code> position.</p>
<p>To approximate the surface area, count the number of sides of each cube that are not immediately connected to another cube. So, if your scan were only two adjacent cubes like <code>1,1,1</code> and <code>2,1,1</code>, each cube would have a single side covered and five sides exposed, a total surface area of <code><em>10</em></code> sides.</p>
<p>Here's a larger example:</p>
<pre><code>2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
</code></pre>
<p>In the above example, after counting up all the sides that aren't connected to another cube, the total surface area is <code><em>64</em></code>.</p>
<p><em>What is the surface area of your scanned lava droplet?</em></p>
</article>


In [29]:
from collections.abc import Iterator


tests = [
    {
        "name": "Example 1",
        "s": """
            1,1,1
            2,1,1
        """,
        "expected": 10,
    },
    {
        "name": "Example 2",
        "s": """
            2,2,2
            1,2,2
            3,2,2
            2,1,2
            2,3,2
            2,2,1
            2,2,3
            2,2,4
            2,2,6
            1,2,5
            3,2,5
            2,1,5
            2,3,5
        """,
        "expected": 64,
    },
]


@dataclass(frozen=True)
class Cube:
    x: int
    y: int
    z: int

    def neigbors(self) -> Iterator[Cube]:
        yield Cube(self.x + 1, self.y, self.z)
        yield Cube(self.x, self.y + 1, self.z)
        yield Cube(self.x, self.y, self.z + 1)
        yield Cube(self.x - 1, self.y, self.z)
        yield Cube(self.x, self.y - 1, self.z)
        yield Cube(self.x, self.y, self.z - 1)


class Cubes:
    def __init__(self, s: str) -> None:
        self.cubes = {
            Cube(*list(map(int, re.findall(r"\d+", t)))) for t in s.strip().splitlines()
        }

    def sides_exposed(self) -> int:
        sides_exposed = 6 * len(self.cubes)
        for cube in self.cubes:
            for neigbor in cube.neigbors():
                if neigbor in self.cubes:
                    sides_exposed -= 1

        return sides_exposed

    def plot(self) -> None:
        x_max = max(cube.x for cube in self.cubes)
        y_max = max(cube.y for cube in self.cubes)
        z_max = max(cube.z for cube in self.cubes)
        d = max(x_max, y_max, z_max) + 1

        voxels = np.zeros((d, d, d), dtype=np.bool)

        for cube in self.cubes:
            voxels[cube.x, cube.y, cube.z] = 1

        # Plot
        ax = plt.figure().add_subplot(projection="3d")

        ax.set_xlim(0, d)
        ax.set_ylim(0, d)
        ax.set_zlim(0, d)

        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("z")

        colors = colors = np.empty(voxels.shape + (4,), dtype=np.float32)
        colors[voxels] = [0, 1, 1, 0.9]

        ax.voxels(voxels, facecolors=colors)

        plt.show()
        pass

    def __str__(self) -> str:
        return f"Cubes(cubes={self.cubes})"


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Cubes(s).sides_exposed()


Test Example 1 passed, for test_part_I.
Test Example 2 passed, for test_part_I.
Success


In [30]:
with open("../input/day18.txt") as quotient:
    puzzle = quotient.read()

print(f"Part I: {Cubes(puzzle).sides_exposed()}")

Part I: 4450


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4450</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Something seems off about your calculation. The cooling rate depends on exterior surface area, but your calculation also included the surface area of air pockets trapped in the lava droplet.</p>
<p>Instead, consider only cube sides that could be reached by the water and steam as the lava droplet tumbles into the pond. The steam will expand to reach as much as possible, completely displacing any air on the outside of the lava droplet but never expanding diagonally.</p>
<p>In the larger example above, exactly one cube of air is trapped within the lava droplet (at <code>2,2,5</code>), so the exterior surface area of the lava droplet is <code><em>58</em></code>.</p>
<p><em>What is the exterior surface area of your scanned lava droplet?</em></p>
</article>


In [31]:
from collections import deque

from more_itertools import minmax


tests = [
    {
        "name": "Example 1",
        "s": """
            2,2,2
            1,2,2
            3,2,2
            2,1,2
            2,3,2
            2,2,1
            2,2,3
            2,2,4
            2,2,6
            1,2,5
            3,2,5
            2,1,5
            2,3,5
        """,
        "expected": 58,
    },
]


class CubesII(Cubes):
    def exterior_surface_area(self) -> int:
        x_min, x_max = minmax(cube.x for cube in self.cubes)
        y_min, y_max = minmax(cube.y for cube in self.cubes)
        z_min, z_max = minmax(cube.z for cube in self.cubes)
        x_min, x_max = x_min - 1, x_max + 1
        y_min, y_max = y_min - 1, y_max + 1
        z_min, z_max = z_min - 1, z_max + 1

        exterior_surface_area = 0

        seen = set()

        queue = deque([Cube(x_min, y_min, z_min)])

        while queue:
            cube = queue.popleft()

            if not (
                x_min <= cube.x <= x_max
                and y_min <= cube.y <= y_max
                and z_min <= cube.z <= z_max
            ):
                continue

            if cube in self.cubes:
                exterior_surface_area += 1
                continue

            if cube in seen:
                continue

            seen.add(cube)

            queue.extend(n for n in cube.neigbors())

        return exterior_surface_area


@test(tests=tests)
def test_part_II(s: str) -> int:
    return CubesII(s).exterior_surface_area()


Test Example 1 passed, for test_part_II.
Success


In [32]:
print(f"Part II: {CubesII(puzzle).exterior_surface_area():,}")

Part II: 2,564


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>2564</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
